In [127]:
import voxel3d as v3d
import plane_fitting as pf
import numpy as np
import pandas as pd
import importlib
importlib.reload(pf)

<module 'plane_fitting' from '/Users/matdou/Documents/Rough-Terrain-Planning/plane_fitting.py'>

In [64]:
# The following paths require the "testing data" to be at the same level as the script
prefix = '2022-07-22_flight__0254_1658494839082804823'
lidar_dir = 'goose_3d_val/lidar/val/2022-07-22_flight'
labels_dir = 'goose_3d_val/labels/val/2022-07-22_flight'
csv_file = 'goose_3d_val/goose_label_mapping.csv'

lidar_data, lidar_labels, label_metadata = pf.load_pointcloud_and_labels(prefix, lidar_dir, labels_dir, csv_file)


# Vertical Z-threshold

In [65]:
z_threshold = 1
#pointcloud, labels = pf.apply_threshold(lidar_data, lidar_labels, z_threshold)
pointcloud = lidar_data
labels = lidar_labels

In [66]:
voxel_labels, voxel_map = v3d.voxelize_point_cloud_2d(pointcloud, voxel_size=1)

In [67]:
n_insufficient_voxels = 0
for i in np.unique(voxel_labels):
    if np.sum(voxel_labels == i) < 4:
        n_insufficient_voxels += 1

print(f"Number of voxels with less than 4 points: {n_insufficient_voxels}")
print(f"Proportion of voxels with less than 4 points: {n_insufficient_voxels / len(np.unique(voxel_labels))}")
#TODO : Maybe remove when len(points) == 3 because it's not enough to compute residuals

Number of voxels with less than 4 points: 2102
Proportion of voxels with less than 4 points: 0.358948087431694


In [68]:
pf.visualize_selected_points(pointcloud, voxel_labels)
#pf.visualize_selected_points(pointcloud, labels, label_metadata)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# Compute residuals indifferently for all classes

In [69]:
voxel_planes, rmse = pf.compute_voxel_planes(pointcloud, voxel_labels)   

Error fitting plane for voxel 0: At least 3 points are required to fit a plane
Error fitting plane for voxel 1: At least 3 points are required to fit a plane
Error fitting plane for voxel 2: At least 3 points are required to fit a plane
Error fitting plane for voxel 3: At least 3 points are required to fit a plane
Error fitting plane for voxel 4: At least 3 points are required to fit a plane
Error fitting plane for voxel 5: At least 3 points are required to fit a plane
Error fitting plane for voxel 6: At least 3 points are required to fit a plane
Error fitting plane for voxel 7: At least 3 points are required to fit a plane
Error fitting plane for voxel 9: At least 3 points are required to fit a plane
Error fitting plane for voxel 10: At least 3 points are required to fit a plane
Error fitting plane for voxel 11: At least 3 points are required to fit a plane
Error fitting plane for voxel 12: At least 3 points are required to fit a plane
Error fitting plane for voxel 14: At least 3 poin

In [70]:
pf.plot_voxel_map(voxel_map, rmse, save_and_open=True, output_file='high_res_voxel_map.png', dpi=300)

# Per label residuals : Work In Progress

In [141]:
data = []
pointcloud = lidar_data

for i in np.unique(lidar_labels):

    # Filter the point cloud based on the current label
    filtered_pointcloud = pointcloud[lidar_labels == i]
    
    # Voxelize the point cloud
    voxel_labels_, voxel_map_ = v3d.voxelize_point_cloud_2d(filtered_pointcloud, voxel_size=30)
    
    # Compute the plane for each voxel and the associated RMSE
    voxel_planes_, rmse_ = pf.compute_voxel_planes(filtered_pointcloud, voxel_labels_)

    # Skip if no RMSE data is available
    if len(rmse_) == 0:
        continue
    
    # Compute the median RMSE for the current label
    median_rmse_ = np.median(list(rmse_.values()))
    
    total_points = 0
    weighted_rmse_sum = 0
    rmse_values = list(rmse_.values())
    
    # Compute RMSE statistics and total points
    for voxel_label, rmse_value in rmse_.items():
        num_points_in_voxel = np.sum(voxel_labels_ == voxel_label)
        weighted_rmse_sum += rmse_value * num_points_in_voxel
        total_points += num_points_in_voxel

    # Average RMSE weighted by the number of points in each voxel
    average_rmse_ = weighted_rmse_sum / total_points
    
    # Calculate additional statistics
    number_of_points_ = len(filtered_pointcloud)
    number_of_voxels_ = len(np.unique(voxel_labels_))
    rmse_sum_ = np.sum(rmse_values)
    rmse_std_ = np.std(rmse_values)
    max_rmse_voxel_ = max(rmse_, key=rmse_.get)  # Voxel with the highest RMSE
    min_rmse_voxel_ = min(rmse_, key=rmse_.get)  # Voxel with the lowest RMSE
    
    # Filter voxels with at least one point
    filtered_voxels_with_points_ = len([v for v in np.unique(voxel_labels_) if np.sum(voxel_labels_ == v) > 0])

    # Append results to the list
    data.append({
        'label': i,
        'RMSE': rmse_,
        'voxel_map': voxel_map_,
        'voxel_labels': voxel_labels_,
        'number_of_points': number_of_points_,
        'number_of_voxels': number_of_voxels_,
        'average_rmse': average_rmse_,
        'median_rmse': median_rmse_,
        'rmse_sum': rmse_sum_,
        'rmse_std': rmse_std_,
        'max_rmse_voxel': max_rmse_voxel_,
        'min_rmse_voxel': min_rmse_voxel_,
        'filtered_voxels_with_points': filtered_voxels_with_points_,
        'total_points_per_voxel': total_points,
        'pointcloud': filtered_pointcloud
    })

# Convert the list into a pandas DataFrame
df = pd.DataFrame(data)


Error fitting plane for voxel 0: At least 3 points are required to fit a plane
Error fitting plane for voxel 3: At least 3 points are required to fit a plane
Error fitting plane for voxel 4: At least 3 points are required to fit a plane
Error fitting plane for voxel 5: At least 3 points are required to fit a plane
Error fitting plane for voxel 2: At least 3 points are required to fit a plane
Error fitting plane for voxel 10: At least 3 points are required to fit a plane
Error fitting plane for voxel 15: At least 3 points are required to fit a plane
Error fitting plane for voxel 0: At least 3 points are required to fit a plane
Error fitting plane for voxel 6: At least 3 points are required to fit a plane
Error fitting plane for voxel 12: At least 3 points are required to fit a plane
Error fitting plane for voxel 14: At least 3 points are required to fit a plane
Error fitting plane for voxel 19: At least 3 points are required to fit a plane
Error fitting plane for voxel 22: At least 3 po

In [143]:
# Assuming 'df' is your residuals DataFrame from previous computations
residuals_df = df

# Perform an inner join (merge) with label_metadata on 'label' (equivalent to 'label_key')
merged_df = pd.merge(label_metadata, residuals_df, left_on='label_key', right_on='label')

# Drop unnecessary columns 'has_instance' and 'hex'
merged_df = merged_df.drop(columns=['has_instance', 'hex'])

# Select and reorder the desired columns
merged_df = merged_df[['label', 'class_name', 'number_of_points', 'average_rmse', 'median_rmse', 'number_of_voxels', 
                       'rmse_std', 'rmse_sum', 'max_rmse_voxel', 'min_rmse_voxel']]

# Sort the merged dataframe by 'median_rmse' in ascending order
sorted_merged_df = merged_df.sort_values(by='median_rmse', ascending=True)

# Output the sorted DataFrame
print(sorted_merged_df)


    label   class_name  number_of_points  average_rmse  median_rmse  \
5      23      asphalt             11837      0.032811     0.014480   
4      22         curb                60      0.013416     0.015329   
1       4     obstacle               142      0.490889     0.026958   
2       8  ego_vehicle               350      0.038699     0.036616   
10     50    low_grass             10751      0.065931     0.052970   
9      41        fence             11253      0.058550     0.057733   
0       0    undefined                26      0.143579     0.097801   
8      38     building               431      1.323684     0.131808   
7      31         soil              1551      0.193333     0.166896   
11     51   high_grass             25143      0.336403     0.235861   
3      17         bush              6835      0.810554     0.519747   
12     59        hedge            116771      0.865780     0.680745   
6      27   tree_crown             15436      1.028776     0.809590   

    n

In [145]:
label =  23
asphalt_points = pointcloud[lidar_labels == label]
voxel_labels_, voxel_map_ = v3d.voxelize_point_cloud_2d(asphalt_points, voxel_size=30)
pf.visualize_selected_points(asphalt_points, voxel_labels_)

In [ ]:
#pf.visualize_selected_points(lidar_data, lidar_labels, label_metadata)